# CNN

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import cv2
import os
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import torchvision
import csv
####################################
def imshow(img):
    plt.imshow(cv2.cvtColor(np.transpose(np.array(img),(1,2,0)),cv2.COLOR_RGB2BGR))
###################################
###################################
#hyperparameters
epochs=10
batch_size=64
learning_rate=0.001
####################################
#load csv and remove the path
with open('Data\driving_log.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))
####################################
for i in range(len(data)):
    for j in range(3):
        data[i][j]=data[i][j].replace('C:\\Users\\mosta\\Downloads\\AI racing team\\beta_simulator_windows\\Data\\IMG\\','')
####################################
#load images in data
directory_images="Data\IMG"
print(len(data))
for i in range(len(data)):
    for j in range(3):
        data[i][j]=transforms.ToTensor()(cv2.imread(directory_images+"\\"+data[i][j]))
        #print(data[i][j].shape)
###################################
#tranform column 3,4,5,6 to tensors
for i in range(len(data)):
    for j in range(3,7):
        data[i][j]=t.tensor(float(data[i][j]))  
###################################
#split data to train,test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
###################################
train_data=t.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_data=t.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=True)
###################################

class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1=nn.Conv2d(3,64,7)
        self.conv2=nn.Conv2d(64,64,5)
        self.conv3=nn.Conv2d(64,64,3)
        self.pool=nn.MaxPool2d(2,2)


        self.fc1=nn.Linear(64*17*37,128)
        self.fc2=nn.Linear(128,2)

    def forward(self,x):
        # N,3,160,320
        #print(x.shape)
        x=self.pool(F.relu(self.conv1(x)))    # N,64,77,157
        #print(x.shape)

        x=self.pool(F.relu(self.conv2(x)))  # N 64,36,76
        #print(x.shape)

        x=self.pool(F.relu(self.conv3(x)))  # N,64,17,37
        #print(x.shape)

        x=x.view(-1,64*17*37)     # N,64*17*37
        #print(x.shape)

        x=F.relu(self.fc1(x))    # N,128
        #print(x.shape)
        
        x=self.fc2(x)            # N,2
        #print(x.shape)
        return x
###################################
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Define the layers for your CNN architecture
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 40 * 80, 64)  # Adjust the input size based on your image dimensions
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)  # Output two values

    def forward(self, x):
        # Define the forward pass of your model
        #N,3,160,320
        x = self.pool1(self.relu1(self.conv1(x))) # N,16,80,160
        x = self.pool2(self.relu2(self.conv2(x))) # N,32,40,80
        x = x.view(-1,32*40*80)  # Flatten the feature maps  #N,32*40*80
        x = self.relu3(self.fc1(x)) # N,64
        x = self.fc2(x) # N,2
        return x

2453


# Train

In [11]:
###################################
model = ConvNet()
#model=CustomCNN()
#print(model)

###################################
criterion=nn.MSELoss()
optimizer=t.optim.Adam(model.parameters(),lr=learning_rate)

n_total_steps = len(train_data)

###################################
for epoch in range(epochs):

    model.train()
    running_loss=0.0

    for i,(center,left,right,steering,thrust,reverse,speed) in enumerate(train_data):
       
        labels=t.cat((steering.unsqueeze(1),thrust.unsqueeze(1)),dim=1)    
        outputs=model(center)
        loss=criterion(outputs,labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss+=loss.item()

        print(f'[{epoch + 1}] loss: {running_loss / n_total_steps:.3f}')

print('Finished Training')
PATH = './final simulation/ARL-Workshop-23-DL-Project/model.pth'
t.save(model, PATH)


[1] loss: 0.011
[1] loss: 0.377
[1] loss: 0.383
[1] loss: 0.387
[1] loss: 0.397
[1] loss: 0.400
[1] loss: 0.406
[1] loss: 0.412
[1] loss: 0.416
[1] loss: 0.419
[1] loss: 0.425
[1] loss: 0.428
[1] loss: 0.431
[1] loss: 0.434
[1] loss: 0.438
[1] loss: 0.441
[1] loss: 0.445
[1] loss: 0.448
[1] loss: 0.450
[1] loss: 0.452
[1] loss: 0.457
[1] loss: 0.460
[1] loss: 0.463
[1] loss: 0.465
[1] loss: 0.469
[1] loss: 0.472
[1] loss: 0.474
[1] loss: 0.477
[1] loss: 0.479
[1] loss: 0.482
[1] loss: 0.485
[2] loss: 0.002
[2] loss: 0.005
[2] loss: 0.007
[2] loss: 0.009
[2] loss: 0.011
[2] loss: 0.013
[2] loss: 0.015
[2] loss: 0.017
[2] loss: 0.020
[2] loss: 0.022
[2] loss: 0.024
[2] loss: 0.025
[2] loss: 0.028
[2] loss: 0.030
[2] loss: 0.032
[2] loss: 0.034
[2] loss: 0.035
[2] loss: 0.037
[2] loss: 0.039
[2] loss: 0.040
[2] loss: 0.042
[2] loss: 0.044
[2] loss: 0.046
[2] loss: 0.047
[2] loss: 0.049
[2] loss: 0.051
[2] loss: 0.052
[2] loss: 0.054
[2] loss: 0.055
[2] loss: 0.056
[2] loss: 0.058
[3] loss

# Test

In [12]:
model.eval()
with t.no_grad():
    for i,(center,left,right,steering,thrust,reverse,speed) in enumerate(test_data):
        labels=t.cat((steering.unsqueeze(1),thrust.unsqueeze(1)),dim=1)    
        outputs=model(center)
        loss=criterion(outputs,labels)

        print(f'[{i + 1}] loss: {loss.item():.3f}')

[1] loss: 0.031
[2] loss: 0.021
[3] loss: 0.019
[4] loss: 0.023
[5] loss: 0.013
[6] loss: 0.035
[7] loss: 0.026
[8] loss: 0.025
